# 📂 Подготовка структуры проекта в Google Colab

В этом блоке я:

- Клонирую репозиторий с GitHub в рабочее пространство Google Colab, чтобы все изменения и новый код автоматически сохранялись в облачном репозитории.
- Перехожу в директорию проекта (все храню на google диске), чтобы всё, что буду создавать дальше, находилось в одной организованной папке.
- Создаю стандартную структуру каталогов для удобства работы над проектом:
  - `src/etl` — для скриптов по сбору и обработке данных (ETL: Extract, Transform, Load)
  - `data/raw` — для хранения исходных (сырых) данных, полученных из hh.ru
  - `models` — для хранения файлов обученных моделей машинного обучения
  - `notebooks` — для Jupyter/Colab-ноутбуков, в которых буду проводить анализ, эксперименты и демонстрацию результатов



In [ ]:
!git clone https://github.com/Annut04ka/hh-hr-bot.git /content/drive/MyDrive/hh-hr-bot
%cd /content/drive/MyDrive/hh-hr-bot
!mkdir -p src/etl data/raw models notebooks

Cloning into '/content/drive/MyDrive/hh-hr-bot'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.
/content/drive/MyDrive/hh-hr-bot


# 📝 В этой части кода я создаю файл requirements.txt

Здесь формируется стандартный список всех библиотек Python, необходимых для запуска и работы нашего проекта.

- Такой файл нужен для воспроизводимости: любой человек или сервис, скачавший наш проект с GitHub, сможет одной командой установить все нужные зависимости.
- Это важно для быстрой настройки окружения в Google Colab, на локальном компьютере, или при автоматическом развёртывании на сервере.

Все библиотеки перечисляются по одной в строке — теперь нам не нужно вспоминать, что ставить, а проверяющий или коллега сможет легко воспроизвести нашу среду!


In [ ]:
%%writefile requirements.txt
pandas
duckdb
lxml
beautifulsoup4
requests
tqdm
python-dotenv

Writing requirements.txt


In [ ]:
# УДАЛИТЬ
!git add requirements.txt
!git commit -m "init: requirements"
!git push origin main

In [ ]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [ ]:
# В этой части кода мы создаём файл .env.example — пример для конфигурации проекта.
# Реальные значения (токены и пароли) НЕ публикуем, только шаблон!
%%writefile .env.example
TELEGRAM_TOKEN=your_telegram_token_here
SUPABASE_URL=https://xyzcompany.supabase.co
SUPABASE_KEY=your_supabase_key_here

Overwriting .env.example


In [ ]:
# В этой части кода мы устанавливаем все необходимые библиотеки,
# чтобы Python-код мог их импортировать и выполнять.
!pip install duckdb lxml beautifulsoup4 requests tqdm python-dotenv

In [ ]:
# В этой части кода мы переходим в папку проекта на Google Диске,
# чтобы все файлы сохранялись именно там и не терялись после отключения Colab.
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [ ]:
# В этой части кода мы собираем вакансии через API hh.ru:
# сначала выгружаем id, потом для каждой вакансии берём подробную информацию
# (description, key_skills и др.), и сохраняем результат в csv и DuckDB.
%%writefile src/etl/fetch_hh.py
import requests
import time
import pandas as pd
import duckdb
from tqdm import tqdm
from pathlib import Path

Path("data/raw").mkdir(parents=True, exist_ok=True)

def fetch_vacancy_ids(pages=3, per_page=100, area="113"):
    """Собирает id вакансий по поиску"""
    vac_ids = []
    for page in tqdm(range(pages), desc="Выгружаем id"):
        url = "https://api.hh.ru/vacancies"
        params = dict(area=area, per_page=per_page, page=page, text="*")
        r = requests.get(url, params=params)
        for item in r.json()["items"]:
            vac_ids.append(item["id"])
        time.sleep(0.2)
    return vac_ids

def fetch_full_vacancies(vac_ids):
    """Детально собирает вакансии по id (description, key_skills и др.)"""
    all_rows = []
    for vid in tqdm(vac_ids, desc="Грузим вакансии по id"):
        url = f"https://api.hh.ru/vacancies/{vid}"
        v = requests.get(url).json()
        all_rows.append({
            "id": v["id"],
            "title": v["name"],
            "published_at": v["published_at"],
            "description": v.get("description", ""),
            "salary_from": (v["salary"] or {}).get("from") if v.get("salary") else None,
            "salary_to": (v["salary"] or {}).get("to") if v.get("salary") else None,
            "salary_currency": (v["salary"] or {}).get("currency") if v.get("salary") else None,
            "experience_hh": (v.get("experience") or {}).get("name"),
            "area_id": int(v["area"]["id"]),
            "skills_raw": ", ".join(s["name"] for s in v.get("key_skills", []))
        })
        time.sleep(0.05)  # не перегружаем hh.ru
    return pd.DataFrame(all_rows)

if __name__ == "__main__":
    # 3 страницы по 100 = 300 вакансий (можно увеличить до лимита)
    vac_ids = fetch_vacancy_ids(pages=3, per_page=100, area="113")
    df = fetch_full_vacancies(vac_ids)
    # Сохраняем в csv
    df.to_csv("data/raw/sample_vacancies.csv", index=False)
    # Сохраняем в DuckDB
    con = duckdb.connect("data/hh.duckdb")
    con.execute("CREATE OR REPLACE TABLE vacancy AS SELECT * FROM df")
    con.close()
    print(f"Сохранили {len(df)} вакансий в data/raw/sample_vacancies.csv и data/hh.duckdb")

Overwriting src/etl/fetch_hh.py


In [ ]:
# В этой части кода мы запускаем наш скрипт для выгрузки вакансий через API hh.ru.
!python src/etl/fetch_hh.py

Выгружаем id: 100% 3/3 [00:04<00:00,  1.53s/it]
Грузим вакансии по id: 100% 300/300 [03:06<00:00,  1.61it/s]
Сохранили 300 вакансий в data/raw/sample_vacancies.csv и data/hh.duckdb


In [ ]:
# В этой части кода мы читаем выгруженный CSV-файл и смотрим, как выглядят наши данные.
import pandas as pd
df = pd.read_csv('data/raw/sample_vacancies.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               300 non-null    int64  
 1   title            300 non-null    object 
 2   published_at     300 non-null    object 
 3   description      300 non-null    object 
 4   salary_from      174 non-null    float64
 5   salary_to        103 non-null    float64
 6   salary_currency  185 non-null    object 
 7   experience_hh    300 non-null    object 
 8   area_id          300 non-null    int64  
 9   skills_raw       158 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 23.6+ KB


In [ ]:
# В этой части кода мы создаём таблицу навыков и связь между вакансиями и навыками.
import duckdb

con = duckdb.connect("data/hh.duckdb")

# Используем только name как ключ (id не нужен для справочника навыков)
con.execute("CREATE TABLE IF NOT EXISTS skill(name TEXT PRIMARY KEY)")

skills = con.execute("SELECT skills_raw FROM vacancy").fetch_df()
uniq_skills = sorted({s.strip() for row in skills.skills_raw for s in str(row).split(",") if s and s.strip()})

for n in uniq_skills:
    con.execute("INSERT INTO skill(name) VALUES (?) ON CONFLICT DO NOTHING", [n])

# Создаём связь vacancy_skill (многие-ко-многим)
# con.execute("DROP TABLE IF EXISTS vacancy_skill")
con.execute("CREATE TABLE IF NOT EXISTS vacancy_skill(vacancy_id BIGINT, skill_name TEXT, PRIMARY KEY(vacancy_id, skill_name))")

for row in con.execute("SELECT id, skills_raw FROM vacancy").fetchall():
    vid, raw = row
    try:
        vid = int(vid)
    except Exception:
        print("⚠️ Некорректный id:", vid, "RAW:", raw)
        continue
    for sk in [s.strip() for s in str(raw).split(",") if s and s.strip()]:
        try:
            con.execute("INSERT OR IGNORE INTO vacancy_skill VALUES (?, ?)", [vid, sk])
        except Exception as e:
            print(f"Ошибка при вставке: {vid}, {sk}, {e}")

con.commit(); con.close()

In [ ]:
# В этой части кода мы быстро смотрим на статистику вакансий и популярных навыков.
import duckdb

con = duckdb.connect("data/hh.duckdb")
print("Всего вакансий:", con.execute("SELECT COUNT(*) FROM vacancy").fetchone()[0])
print("Всего уникальных навыков:", con.execute("SELECT COUNT(*) FROM skill").fetchone()[0])

# Топ-10 популярных навыков
print(con.execute("""
SELECT s.name, COUNT(*) as n FROM skill s
JOIN vacancy_skill vs ON s.id=vs.skill_id
GROUP BY s.name ORDER BY n DESC LIMIT 10
""").fetchdf())

con.close()

In [ ]:
# В этой части кода мы проводим первичный анализ (EDA) по базе вакансий и навыков.

import duckdb

# Подключаемся к базе данных
con = duckdb.connect("data/hh.duckdb")

# 1. Сколько всего вакансий в базе?
num_vacancies = con.execute("SELECT COUNT(*) FROM vacancy").fetchone()[0]
print(f"Всего вакансий: {num_vacancies}")

# 2. Сколько уникальных навыков в справочнике?
num_skills = con.execute("SELECT COUNT(*) FROM skill").fetchone()[0]
print(f"Всего уникальных навыков: {num_skills}")

# 3. ТОП-10 самых востребованных навыков (по числу вакансий)
print("\nТОП-10 востребованных навыков:")
top_skills = con.execute("""
    SELECT skill_name, COUNT(*) AS n
    FROM vacancy_skill
    GROUP BY skill_name
    ORDER BY n DESC
    LIMIT 10
""").fetchdf()
print(top_skills)

# 4. Распределение вакансий по городам (по area_id)
print("\nРаспределение вакансий по городам (area_id):")
city_dist = con.execute("""
    SELECT area_id, COUNT(*) as n
    FROM vacancy
    GROUP BY area_id
    ORDER BY n DESC
    LIMIT 10
""").fetchdf()
print(city_dist)

con.close()


Всего вакансий: 300
Всего уникальных навыков: 526

ТОП-10 востребованных навыков:
                     skill_name   n
0               Деловое общение  17
1              Работа в команде  16
2  Знание устройства автомобиля  12
3                Грамотная речь  12
4             Деловая переписка  11
5           Ручное тестирование  11
6           Стрессоустойчивость  11
7               Пользователь ПК  10
8        Организаторские навыки  10
9               Ответственность   9

Распределение вакансий по городам (area_id):
   area_id    n
0        1  175
1        2   18
2       53    8
3       99    6
4       78    6
5        4    6
6       11    5
7       26    4
8       66    4
9       76    3


In [ ]:
%cd /content/drive/MyDrive/hh-hr-bot
!git pull origin main


/content/drive/MyDrive/hh-hr-bot
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 1.85 KiB | 55.00 KiB/s, done.
From https://github.com/Annut04ka/hh-hr-bot
 * branch            main       -> FETCH_HEAD
   3c736c0..0f92aa4  main       -> origin/main
Updating 3c736c0..0f92aa4
Fast-forward
 README.md | 34 +++++++++++++++++++++++++++++++++-
 1 file changed, 33 insertions(+), 1 deletion(-)
